__Instead of predicting the whole month, I tried recursive/walk forward prediction method, and also residual modeling. The function below simply predicts one day(24 hours) using the past data, then adds that predicted day into training data and retrains the model to predict the next day, iteratively.__
* __Step size must be divisible with test size. Below, step size 24 is one day, means the function will iterate 744/24 steps which is 31 days.__

* __Residuals of X, y are modeled, predicted, and those predictions of residuals are also modeled. That means this function predicts the residuals of residuals at every step. Mean of those predictions added to original prediction to naively fix the bias.__

* __Function also outputs the last element of training set and range of test set. This allows user to check every iteration.__

* __At the end function returns a serie with predictions__

In [10]:
def walk_forward_predict(X, y, test_size=24*31, steps=24):
    
    train_size = len(X) - test_size
    predictions = pd.DataFrame({'predictions': [0] * test_size})
    
    print(f'Step size : {steps} --- test size : {test_size} --- total steps : {int(test_size/steps)}\n\n')
    
    for i in range(int(test_size/steps)):

        X_train = X[:train_size + (i*steps)]
        y_train = y[:train_size + (i*steps)]

        X_val = X[train_size + (i*steps):train_size + ((i+1)*steps)]
        
        print(f'Step : {i+1} - train last index: {X_train.index.max()}\n')
        print(f'test index range: {X_val.index.min()} to {X_val.index.max()}\n')
        
        model1 = CatBoostRegressor(**params_izmir_cat,random_state=42,thread_count=-1,silent=True)
        model1.fit(X_train, y_train)
        y_train_pred = model1.predict(X_train)
        res1 = y_train - y_train_pred
        
        model2 = CatBoostRegressor(**params_izmir_cat,random_state=42,thread_count=-1,silent=True)
        model2.fit(X_train, res1)
        y_train_pred2 = model2.predict(X_train)
        X_res = y_train_pred + y_train_pred2
        res2 = y_train - X_res
        
        model3 = CatBoostRegressor(**params_izmir_cat,random_state=42,thread_count=-1,silent=True)
        model3.fit(X_train, res2)
        
        
        y_pred = model1.predict(X_val) + np.mean(model2.predict(X_val)) + np.mean(model3.predict(X_val))

        predictions.loc[i*steps:((i+1)*steps)-1, 'predictions'] = y_pred
        
    print(f'\n\n !!! DONE !!!')
    
    return predictions

pred_izmir_cat = walk_forward_predict(X_izmir, y_izmir, test_size=24*31, steps=24)

Step size : 24 --- test size : 744 --- total steps : 31


Step : 1 - train last index: 2022-07-31 23:00:00

test index range: 2022-08-01 00:00:00 to 2022-08-01 23:00:00

Step : 2 - train last index: 2022-08-01 23:00:00

test index range: 2022-08-02 00:00:00 to 2022-08-02 23:00:00

Step : 3 - train last index: 2022-08-02 23:00:00

test index range: 2022-08-03 00:00:00 to 2022-08-03 23:00:00

Step : 4 - train last index: 2022-08-03 23:00:00

test index range: 2022-08-04 00:00:00 to 2022-08-04 23:00:00

Step : 5 - train last index: 2022-08-04 23:00:00

test index range: 2022-08-05 00:00:00 to 2022-08-05 23:00:00

Step : 6 - train last index: 2022-08-05 23:00:00

test index range: 2022-08-06 00:00:00 to 2022-08-06 23:00:00

Step : 7 - train last index: 2022-08-06 23:00:00

test index range: 2022-08-07 00:00:00 to 2022-08-07 23:00:00

Step : 8 - train last index: 2022-08-07 23:00:00

test index range: 2022-08-08 00:00:00 to 2022-08-08 23:00:00

Step : 9 - train last index: 2022-08-08 23:00:

In [171]:
pred_izmir_cat.isna().sum()

predictions    0
dtype: int64

In [172]:
pred_izmir_cat.min()

predictions    1231.730859
dtype: float64

In [173]:
pred_izmir_cat.max()

predictions    2681.523386
dtype: float64

__If we choose the step size same as the set size like below, it becomes a direct prediction instead of a recursive prediction, and process will be finished in one step just like a regular prediction.__

In [181]:
def walk_forward_predict(X, y, test_size=24*31, steps=24):
    
    train_size = len(X) - test_size
    predictions = pd.DataFrame({'predictions': [0] * test_size})
    
    print(f'Step size : {steps} --- test size : {test_size} --- total steps : {int(test_size/steps)}\n\n')
    
    for i in range(int(test_size/steps)):

        X_train = X[:train_size + (i*steps)]
        y_train = y[:train_size + (i*steps)]

        X_val = X[train_size + (i*steps):train_size + ((i+1)*steps)]
        
        print(f'Step : {i+1} - train last index: {X_train.index.max()}\n')
        print(f'test index range: {X_val.index.min()} to {X_val.index.max()}\n')
        
        model1 = CatBoostRegressor(**params_manisa_cat,random_state=42,thread_count=-1,silent=True)
        model1.fit(X_train, y_train)
        y_train_pred = model1.predict(X_train)
        res1 = y_train - y_train_pred
        
        model2 = CatBoostRegressor(**params_manisa_cat,random_state=42,thread_count=-1,silent=True)
        model2.fit(X_train, res1)
        y_train_pred2 = model2.predict(X_train)
        X_res = y_train_pred + y_train_pred2
        res2 = y_train - X_res
        
        model3 = CatBoostRegressor(**params_manisa_cat,random_state=42,thread_count=-1,silent=True)
        model3.fit(X_train, res2)
        
        
        y_pred = model1.predict(X_val) + np.mean(model2.predict(X_val)) + np.mean(model3.predict(X_val))

        predictions.loc[i*steps:((i+1)*steps)-1, 'predictions'] = y_pred
    print(f'\n\n !!! DONE !!!')
    return predictions

pred_manisa_cat = walk_forward_predict(X_manisa, y_manisa, test_size=24*31, steps=744)

Step size : 744 --- test size : 744 --- total steps : 1


Step : 1 - train last index: 2022-07-31 23:00:00

test index range: 2022-08-01 00:00:00 to 2022-08-31 23:00:00



 !!! DONE !!!


In [183]:
pred_manisa_cat.isna().sum()

predictions    0
dtype: int64

In [184]:
pred_manisa_cat.min()

predictions    210.929337
dtype: float64

In [185]:
pred_manisa_cat.max()

predictions    458.867937
dtype: float64

__We can also perform cross validation with this method__

In [97]:
def walk_forward_cv(X, y, test_size, steps):
    
    train_size = len(X) - test_size
    predictions = pd.DataFrame({'predictions': [0] * test_size})

    for i in range(int(test_size/steps)):
        X_train = X[:train_size + (i*steps)]
        y_train = y[:train_size + (i*steps)]

        X_val = X[train_size + (i*steps):train_size + ((i+1)*steps)]
        y_val = y[train_size + (i*steps):train_size + ((i+1)*steps)]
        
        model1 = LGBMRegressor(random_state=42,n_jobs=-1,silent=True,verbose=-1)
        model1.fit(X_train, y_train)
        y_train_pred = model1.predict(X_train)
        res1 = y_train - y_train_pred
        
        model2 = LGBMRegressor(random_state=42,n_jobs=-1,silent=True,verbose=-1)
        model2.fit(X_train, res1)
        y_train_pred2 = model2.predict(X_train)
        X_res = y_train_pred + y_train_pred2
        res2 = y_train - X_res
        
        model3 = LGBMRegressor(random_state=42,n_jobs=-1,silent=True,verbose=-1)
        model3.fit(X_train, res2)
        
        
        y_pred = model1.predict(X_val) + np.mean(model2.predict(X_val)) + np.mean(model3.predict(X_val))

        predictions.loc[i*steps:((i+1)*steps)-1, 'predictions'] = y_pred
        
    return predictions

p = walk_forward_cv(X_manisa[:40152], y_manisa[:40152], test_size=24*31, steps=24)